In [9]:
!pip install python-pachyderm
!pip install skl2onnx


[notice] A new release of pip is available: 23.1.2 -> 23.2
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.3/292.3 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 91.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.8/83.8 kB 280.9 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.1.2 -> 23.2
[notice] To update, run: pip install --upgrade pip


In [2]:
commit_id = "be82b14369c74ff7bbdf9866d6f701ee"

In [3]:
import python_pachyderm as pc
client = pc.Client(host="pachd.pachyderm.svc.cluster.local")


In [4]:
import pandas as pd
wines = client.get_file(("wines", "master", commit_id), "wine.csv")
wines_df = pd.read_csv(wines, delimiter=';')


In [5]:
# Split data into training and test

wine_train = wines_df.sample(frac=0.7)
print(wine_train.shape)

(3429, 12)


In [6]:
wine_test = wines_df.loc[~wines_df.set_index(list(wines_df.columns)).index.isin(wine_train.set_index(list(wine_train.columns)).index)]
print(wine_test.shape)

(1088, 12)


In [7]:
from sklearn.linear_model import LinearRegression
features = ['volatile acidity', 'residual sugar', 'chlorides', 'total sulfur dioxide', 'sulphates', 'alcohol']
wine_ln_reg = LinearRegression()
wine_ln_reg = wine_ln_reg.fit(wine_train[features], wine_train['quality'])

In [11]:
import boto3, pickle, os
from skl2onnx import convert_sklearn
import onnx
from skl2onnx.common.data_types import FloatTensorType

# Save the model to a file
with open('model.bin', 'wb') as f:
    pickle.dump(wine_ln_reg, f)
    
# Set up the S3 client
s3 = boto3.client('s3',
                  endpoint_url='http://minio-ml-workshop.minio.svc.cluster.local:9000',
                  aws_access_key_id='minio',
                  aws_secret_access_key=os.getenv("MODEL_REGISTRY_PASSWORD"))

# Upload a file to the bucket
bucket_name = 'wine-models'
object_name = 'v1/model.onnx'
file_path = 'model.onnx'


# Convert the model to ONNX format
initial_type = [('float_input', FloatTensorType([None, 2]))]
onnx_model = convert_sklearn(wine_ln_reg, initial_types=initial_type)

# Save the ONNX model to a file
onnx.save(onnx_model, 'model.onnx')

s3.upload_file(file_path, bucket_name, object_name)

In [ ]:
print(wine_ln_reg.coef_)
print(wine_ln_reg.intercept_)

In [ ]:
wine_ln_pred = wine_ln_reg.predict(wine_test[features])
wine_ln_resid = wine_ln_pred - wine_test['quality']

In [ ]:
import scipy.stats as stats
import matplotlib.pyplot as plt
plt.figure(figsize=(12,8))
# Normal Q-Q plot
stats.probplot(wine_ln_resid, dist="norm", plot=plt)
plt.xlabel("Theoretical Quantiles",fontsize=15)
plt.ylabel("Ordered Values",fontsize=15)
plt.title("Normal Q-Q Plot",fontsize=20)